Encode with the scaled data


In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.decomposition import PCA

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dataset = torch.load('/content/drive/MyDrive/Autoencoding/raw_cv_whiten.pt')

In [ ]:
def lin_block(in_dim,out_dim,bias=True):
    layers = [nn.Linear(in_dim,out_dim,bias= True),
              #nn.BatchNorm2d(out_dim),
              nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    return nn.Sequential(*layers)


class Resnet9(nn.Module):
      def __init__(self, in_dim,hidden_layers):
        super().__init__()
        
        # Encode1
        
        self.lin1 = lin_block(in_dim,hidden_layers[0])         
        self.lin2 = lin_block(hidden_layers[0],hidden_layers[1])
        self.lin3 = lin_block(hidden_layers[1],hidden_layers[2]) 
        
        self.res1 = nn.Sequential(lin_block(hidden_layers[2], hidden_layers[2]),  
                                  lin_block(hidden_layers[2], hidden_layers[2]))
        
        #Encode2
        self.lin4 = lin_block(hidden_layers[2],hidden_layers[1])         
        self.lin5 = lin_block(hidden_layers[1],hidden_layers[0])
        
        self.res2 = nn.Sequential(lin_block(hidden_layers[0], hidden_layers[0]),  
                                  lin_block(hidden_layers[0], hidden_layers[0]))
        self.lin6 = nn.Linear(hidden_layers[0],in_dim)
        
         # Decode1
        
        self.lin7 = lin_block(in_dim,hidden_layers[0])         
        self.lin8 = lin_block(hidden_layers[0],hidden_layers[1])
        self.lin9 = lin_block(hidden_layers[1],hidden_layers[2]) 
        
        self.res3 = nn.Sequential(lin_block(hidden_layers[2], hidden_layers[2]),  
                                  lin_block(hidden_layers[2], hidden_layers[2]))
        
        #Decode2
        self.lin10 = lin_block(hidden_layers[2],hidden_layers[1])         
        self.lin11 = lin_block(hidden_layers[1],hidden_layers[0])
        
        self.res4 = nn.Sequential(lin_block(hidden_layers[0], hidden_layers[0]),  
                                  lin_block(hidden_layers[0], hidden_layers[0]))
        self.lin12 = nn.Linear(hidden_layers[0],in_dim)
        
      def encode(self,in_data):
          out = self.lin1(in_data.float())
          out = self.lin2(out)
          out = self.lin3(out)
          out = self.res1(out)+out
          out = self.lin4(out)
          out = self.lin5(out)
          out = self.res2(out)+out
          out = self.lin6(out)
          return out
           
      def decode(self,lat_data):
          out = self.lin7(lat_data.float())
          out = self.lin8(out)
          out = self.lin9(out)
          out = self.res3(out)+out
          out = self.lin10(out)
          out = self.lin11(out)
          out = self.res4(out)+out
          out = self.lin12(out)
          return out

In [ ]:
hidden_layers = [64,128,256] ; in_dim=2;
Resnet9(in_dim,hidden_layers)

Resnet9(
  (lin1): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (lin2): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (lin3): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (lin4): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (lin5): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): LeakyReLU(negative_sl

In [ ]:
device=torch.device('cuda')
encoder = Resnet9(in_dim,hidden_layers).to(device)

In [ ]:
encoder.load_state_dict(torch.load('/content/drive/MyDrive/DNN_TAE/dnn_tae_state_dict.pt'))

<All keys matched successfully>

In [ ]:
#data = train_dataset[:]

In [ ]:
x= int(4022018/2002)
print(x)

2009


In [ ]:
import gc
gc.collect()

400

In [ ]:
from tqdm.notebook  import tqdm

In [ ]:
who_ls

['PCA',
 'Resnet9',
 'conv_block1',
 'conv_block2',
 'device',
 'drive',
 'encoder',
 'gc',
 'make_2_dim',
 'make_4_dim',
 'nn',
 'np',
 'torch',
 'tqdm',
 'train_dataset',
 'x']

In [ ]:
#lat1 = make_2_dim(encoder.encode(make_4_dim(train_dataset.to(device))))

In [ ]:
%%time
#latent = []
for i in tqdm(range(1,2003)):
    lat1 = (encoder.encode((train_dataset[(i-1)*x:(i*x)].to(device))))
    torch.save(lat1,'lat'+str(i)+'.pt')
    #latent.append(lat1)
    del lat1 
    gc.collect()


CPU times: user 1min 30s, sys: 482 ms, total: 1min 31s
Wall time: 1min 30s


In [ ]:
import glob

In [ ]:
file = sorted(glob.glob('lat*'))

In [ ]:
file b

In [ ]:
X = []
for f in file:
    X.append(torch.load(f))

X1 = torch.tensor(torch.cat(X))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
rm -r lat*

In [ ]:
torch.save(X1,'latent.pt')

In [ ]:
X1.size()

torch.Size([4022018, 2])